# INTRODUCTION
In this notebook we will be using the following packages:
  * [pyspark](https://spark.apache.org/docs/latest/api/python/pyspark.html)
  * [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
  * [pyarrow](https://arrow.apache.org/docs/python/index.html)

We will be using the following data:
  * dressipi_recsys2022

The dataset is split in 4 csv files:
   * candidate_items : contains the candidate items for each user
   * item_features : contains the features for each item (items can have multiple features)
   * train_purchases : contains the purchases for each user
   * train_sessions : contains the sessions for each user



In [95]:
# Uncomment to install the required packages
# %pip install pyspark
# %pip install pyarrow


## Loading the dataset

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np

# start spark session
spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()

candidate_items, item_features, train_purchases, train_sessions = [None] * 4
datasets = [candidate_items, item_features, train_purchases, train_sessions]

#load all datasets
def load_candidate_items():
    global candidate_items
    candidate_items = spark.read.csv("dressipi_recsys2022/candidate_items.csv", header=True)
    candidate_items = candidate_items.withColumn("item_id", candidate_items["item_id"].cast("int"))

def load_item_features():
    global item_features
    item_features = spark.read.csv("dressipi_recsys2022/item_features.csv", header=True)
    item_features = item_features.withColumn("item_id", item_features["item_id"].cast("int"))
    item_features = item_features.withColumn("feature_category_id", item_features["feature_category_id"].cast("int"))
    item_features = item_features.withColumn("feature_value_id", item_features["feature_value_id"].cast("int"))

def load_train_purchases():
    global train_purchases
    train_purchases = spark.read.csv("dressipi_recsys2022/train_purchases.csv", header=True)
    train_purchases = train_purchases.withColumn("session_id", train_purchases["session_id"].cast("int"))
    train_purchases = train_purchases.withColumn("item_id", train_purchases["item_id"].cast("int"))
    train_purchases = train_purchases.withColumn("date", train_purchases["date"].cast("timestamp"))

def load_train_sessions():
    global train_sessions
    train_sessions = spark.read.csv("dressipi_recsys2022/train_sessions.csv", header=True)
    train_sessions = train_sessions.withColumn("session_id", train_sessions["session_id"].cast("int"))
    train_sessions = train_sessions.withColumn("item_id", train_sessions["item_id"].cast("int"))
    train_sessions = train_sessions.withColumn("date", train_sessions["date"].cast("timestamp"))

def load_datasets():
    global datasets
    load_candidate_items()
    load_item_features()
    load_train_purchases()
    load_train_sessions()
    datasets = [candidate_items, item_features, train_purchases, train_sessions]

load_datasets()


## Quick look at the data

### train_sessions

In [97]:
train_sessions.printSchema()
train_sessions.show(5)

root
 |-- session_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- date: timestamp (nullable = true)

+----------+-------+--------------------+
|session_id|item_id|                date|
+----------+-------+--------------------+
|         3|   9655|2020-12-18 21:25:...|
|         3|   9655|2020-12-18 21:19:...|
|        13|  15654|2020-03-13 19:35:...|
|        18|  18316|2020-08-26 19:18:...|
|        18|   2507|2020-08-26 19:16:...|
+----------+-------+--------------------+
only showing top 5 rows



### train_purchases

In [98]:
train_purchases.show(5)

+----------+-------+--------------------+
|session_id|item_id|                date|
+----------+-------+--------------------+
|         3|  15085|2020-12-18 21:26:...|
|        13|  18626|2020-03-13 19:36:...|
|        18|  24911|2020-08-26 19:20:...|
|        19|  12534|2020-11-02 17:16:...|
|        24|  13226|2020-02-26 18:27:...|
+----------+-------+--------------------+
only showing top 5 rows



### item_features

In [99]:
item_features.show(5)

+-------+-------------------+----------------+
|item_id|feature_category_id|feature_value_id|
+-------+-------------------+----------------+
|      2|                 56|             365|
|      2|                 62|             801|
|      2|                 68|             351|
|      2|                 33|             802|
|      2|                 72|              75|
+-------+-------------------+----------------+
only showing top 5 rows



### candidate_items

In [100]:
candidate_items.show(5)

+-------+
|item_id|
+-------+
|      4|
|      8|
|      9|
|     19|
|     20|
+-------+
only showing top 5 rows



# Part 1 : Pipeline

## Checking for null values

In [101]:
# Are there missing values (na or null) in any of those dataframes ? print the number of missing values
print("candidate_items:", candidate_items.filter(candidate_items["item_id"].isNull()).count())
print("item_features:", item_features.filter(item_features["item_id"].isNull()).count())
print("train_purchases:", train_purchases.filter(train_purchases["item_id"].isNull()).count())
print("train_sessions:", train_sessions.filter(train_sessions["item_id"].isNull()).count())

candidate_items: 0
item_features: 0
train_purchases: 0
train_sessions: 0


## Feature engineering

1) Month

In [29]:
month = train_sessions.rdd.map(lambda x: (x["session_id"], int(x["date"].strftime("%m")))).reduceByKey(min)
print(month.take(5))

[(24, 2), (48, 4), (184, 4), (208, 12), (232, 9)]


2) Day of month

In [104]:
day_of_month = train_sessions.rdd.map(lambda x: (x["session_id"], int(x["date"].strftime("%d")))).reduceByKey(min)
print(day_of_month.take(5))

[(24, 26), (48, 15), (184, 4), (208, 13), (232, 11)]


3) Weekday

In [22]:
weekday = train_sessions.rdd.map(lambda x: (x["session_id"], int(x["date"].strftime("%w")))).reduceByKey(min)
print(weekday.take(5))


[(24, 3), (48, 3), (184, 0), (208, 0), (232, 5)]


4) Hour period

In [106]:
hour_period = train_sessions.rdd.map(lambda x: (x["session_id"], int(x["date"].strftime("%H")))).reduceByKey(min)
print(hour_period.take(5))


[(24, 17), (48, 17), (184, 1), (208, 9), (232, 12)]


5) Season (Meteorological)

In [26]:
def get_season(month):
    if month == 12 or month <= 2: return 0
    elif 2 < month <= 5: return 1
    elif 5 < month <= 8: return 2
    elif 8 < month <= 11: return 3

season = train_sessions.rdd.map(lambda x: (x["session_id"], get_season(int(x["date"].strftime("%m"))))).reduceByKey(min)
print(season.take(5))

[(24, 0), (48, 1), (184, 1), (208, 0), (232, 3)]


6) Average time between consecutive item views

In [11]:
def get_average_time(dates):
    import datetime
    dates = sorted(list(dates))
    avgs = [dates[i+1] - dates[i] for i in range(len(dates)-1)]
    return sum(avgs, datetime.timedelta())/len(avgs) if len(avgs) > 0 else datetime.timedelta(0)

average_time = train_sessions.rdd.map(lambda x: (x["session_id"], x["date"])).groupByKey().mapValues(get_average_time)

average_time.take(5)

[(24, datetime.timedelta(seconds=462, microseconds=983375)),
 (48, datetime.timedelta(seconds=657, microseconds=103000)),
 (184, datetime.timedelta(0)),
 (208, datetime.timedelta(0)),
 (232, datetime.timedelta(0))]

7) Number of distinct items

In [4]:
distinct_nb = train_sessions.rdd.map(lambda x: (x["session_id"], x["item_id"])).groupByKey().mapValues(lambda x: len(set(x)))
distinct_nb.take(5)

[(24, 8), (48, 2), (184, 1), (208, 1), (232, 1)]

8) Number of repetitive items

In [108]:
repetitive_nb = train_sessions.rdd.map(lambda x: (x["session_id"], x["item_id"])).groupByKey().mapValues(lambda x: len(list(x)) - len(set(x)))
repetitive_nb.take(5)

[(24, 1), (48, 0), (184, 0), (208, 0), (232, 0)]

9) Same category

In [109]:
item_features_rdd = item_features.rdd.map(lambda x: (x["item_id"], (x["feature_category_id"], x["feature_value_id"]))).groupByKey().mapValues(lambda x: [(a,b) for a, b in x])

item_features_rdd.take(5)

[(2,
  [(56, 365),
   (62, 801),
   (68, 351),
   (33, 802),
   (72, 75),
   (29, 123),
   (16, 38),
   (50, 76),
   (61, 462),
   (53, 6),
   (7, 394),
   (69, 885),
   (47, 123)]),
 (4,
  [(55, 267),
   (17, 378),
   (5, 605),
   (69, 538),
   (18, 289),
   (68, 373),
   (50, 317),
   (73, 544),
   (65, 521),
   (7, 837),
   (59, 180),
   (47, 218),
   (46, 825),
   (56, 365),
   (4, 618),
   (26, 268),
   (3, 793),
   (19, 148),
   (32, 902),
   (63, 861),
   (22, 881),
   (61, 462),
   (45, 559),
   (72, 75)]),
 (8,
  [(56, 365),
   (55, 267),
   (7, 798),
   (69, 592),
   (72, 75),
   (3, 793),
   (45, 559),
   (18, 817),
   (59, 856),
   (73, 544),
   (5, 605),
   (61, 462),
   (68, 351),
   (4, 618),
   (65, 521),
   (26, 268),
   (50, 317),
   (32, 902),
   (46, 825),
   (17, 378),
   (19, 700),
   (11, 735),
   (63, 861),
   (47, 549)]),
 (10,
  [(56, 365),
   (15, 451),
   (63, 816),
   (12, 410),
   (30, 564),
   (61, 706),
   (50, 128),
   (69, 639),
   (7, 798),
   (30, 48

In [124]:
def get_same_category(x):
    dico = dict()
    for item in x:
        for cat in item:
            if cat in dico:
                dico[cat] += 1
            else:
                dico[cat] = 0

    res = 0
    for val in dico.values():
        if val > 0:
            res += 1

    return res

same_category = item_features_rdd.join(train_sessions.rdd.map(lambda x: (x["item_id"], x["session_id"]))).map(lambda x: (x[1][1], x[1][0])).groupByKey().mapValues(get_same_category)
same_category.take(5)

[(41200, 53), (198620, 0), (427520, 56), (639330, 83), (688110, 42)]

10) Different category

In [125]:
def get_different_category(x):
    dico = dict()
    for item in x:
        for cat in item:
            if cat in dico:
                dico[cat] += 1
            else:
                dico[cat] = 0

    res = 0
    for val in dico.values():
        if val == 0:
            res += 1

    return res

diff_category = item_features_rdd.join(train_sessions.rdd.map(lambda x: (x["item_id"], x["session_id"]))).map(lambda x: (x[1][1], x[1][0])).groupByKey().mapValues(get_different_category)
diff_category.take(5)

[(41200, 30), (198620, 24), (427520, 19), (639330, 26), (688110, 47)]

11) Last item

In [14]:
def get_last_item(items):
    return max(list(items), key=lambda i: i[1])[0]

last_item = train_sessions.rdd.map(lambda x: (x["session_id"], (x["item_id"], x["date"]))).groupByKey().mapValues(lambda x: max(x, key=lambda i: i[1])[0])
last_item.take(5)

[(24, 18476), (48, 26404), (184, 14383), (208, 26257), (232, 19464)]

# Part 2 : Feature selection


## Ranking algorithm

In [37]:
# Pearson correlation

def pearson_reduce(a, b):
    a, b = a[1], b[1]

    n = a[0] + b[0]
    x = a[1] + b[1]
    y = a[2] + b[2]
    x2 = a[3] + b[3]
    y2 = a[4] + b[4]
    xy = a[5] + b[5]

    return 0, (n, x, y, x2, y2, xy)

def calculate_pearson(a):
    import math

    n = a[0]
    x = a[1]
    y = a[2]
    x2 = a[3]
    y2 = a[4]
    xy = a[5]

    return (n*xy - x * y) / (math.sqrt((n*x2 - (x**2)) * (n*y2 - (y**2))))

test = season.join(season)

d = test.mapValues(lambda a: (1, a[0], a[1], a[0]**2, a[1]**2, a[0]*a[1])).reduce(pearson_reduce)[1]
print(calculate_pearson(d))

1.0


## Forward feature selection

# Part 3 : Model

In [112]:
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4]

[]
[Row(item_id=2, feature_category_id=56, feature_value_id=365), Row(item_id=2, feature_category_id=62, feature_value_id=801), Row(item_id=2, feature_category_id=68, feature_value_id=351), Row(item_id=2, feature_category_id=33, feature_value_id=802), Row(item_id=2, feature_category_id=72, feature_value_id=75), Row(item_id=2, feature_category_id=29, feature_value_id=123), Row(item_id=2, feature_category_id=16, feature_value_id=38), Row(item_id=2, feature_category_id=50, feature_value_id=76), Row(item_id=2, feature_category_id=61, feature_value_id=462), Row(item_id=2, feature_category_id=53, feature_value_id=6), Row(item_id=2, feature_category_id=7, feature_value_id=394), Row(item_id=2, feature_category_id=69, feature_value_id=885), Row(item_id=2, feature_category_id=47, feature_value_id=123)]
[]
[Row(session_id=3401213, item_id=2, date=datetime.datetime(2021, 3, 28, 8, 24, 15, 321000))]


## Forward feature selection

# Part 3 : Model

In [ ]:
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

In [ ]:
# show rows with item_id == 2 in all dataframes
for d in datasets:
    print(d.filter(d["item_id"] == 2).collect())